In [1]:
from hoboreader import HoboReader

In [2]:
hr=HoboReader('sample_hobo_data.csv')

In [3]:
hr.header_list

[{'column': 0,
  'title': '#',
  'timezone_str': None,
  'units': None,
  'logger_serial_number': None,
  'sensor_serial_number': None},
 {'column': 1,
  'title': 'Date Time',
  'timezone_str': 'GMT+01:00',
  'units': None,
  'logger_serial_number': None,
  'sensor_serial_number': None},
 {'column': 2,
  'title': 'Temp',
  'timezone_str': None,
  'units': 'Â°F',
  'logger_serial_number': '10469238',
  'sensor_serial_number': '10469238'},
 {'column': 3,
  'title': 'Coupler Detached',
  'timezone_str': None,
  'units': None,
  'logger_serial_number': '10469238',
  'sensor_serial_number': None},
 {'column': 4,
  'title': 'Coupler Attached',
  'timezone_str': None,
  'units': None,
  'logger_serial_number': '10469238',
  'sensor_serial_number': None},
 {'column': 5,
  'title': 'Host Connected',
  'timezone_str': None,
  'units': None,
  'logger_serial_number': '10469238',
  'sensor_serial_number': None},
 {'column': 6,
  'title': 'Stopped',
  'timezone_str': None,
  'units': None,
  'logge

In [4]:
hr.datetimes

[datetime.datetime(2019, 10, 10, 11, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'GMT+01:00')),
 datetime.datetime(2019, 10, 10, 11, 0, 2, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'GMT+01:00')),
 datetime.datetime(2019, 10, 10, 11, 5, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'GMT+01:00')),
 datetime.datetime(2019, 10, 10, 11, 10, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'GMT+01:00')),
 datetime.datetime(2019, 10, 10, 11, 15, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'GMT+01:00')),
 datetime.datetime(2019, 10, 10, 11, 20, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'GMT+01:00')),
 datetime.datetime(2019, 10, 10, 11, 25, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'GMT+01:00')),
 datetime.datetime(2019, 10, 10, 11, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600), 'GMT+01:00')),
 datetime.datetime(2019, 10, 10, 11, 35, tzinfo=datetime.timezone(datetime.timed

In [5]:
import pandas as pd

In [6]:
def get_dataframe_column_multiindex(header_list):
    names=[]
    tuples=[]
    for header in hr.header_list:
        for k in header.keys():
            if not k in names and not k=='column': names.append(k)
        t=[]
        for name in names:
            t.append(header.get(name,None))
        tuples.append(tuple(t))
    
    #tuples.append(tuple(['datetimes']+[None]*(len(names)-1)))
    return pd.MultiIndex.from_tuples(tuples,names=names)

multiindex=get_dataframe_column_multiindex(hr.header_list)
multiindex

MultiIndex(levels=[['#', 'Coupler Attached', 'Coupler Detached', 'Date Time', 'End Of File', 'Host Connected', 'Stopped', 'Temp'], ['GMT+01:00'], ['Â°F'], ['10469238'], ['10469238']],
           codes=[[0, 3, 7, 2, 1, 5, 6, 4], [-1, 0, -1, -1, -1, -1, -1, -1], [-1, -1, 0, -1, -1, -1, -1, -1], [-1, -1, 0, 0, 0, 0, 0, 0], [-1, -1, 0, -1, -1, -1, -1, -1]],
           names=['title', 'timezone_str', 'units', 'logger_serial_number', 'sensor_serial_number'])

In [7]:
import numpy as np
def get_dataframe(header_list,data_rows):
    columns=get_dataframe_column_multiindex(header_list)
    index=pd.Index(hr.datetimes, name='datetimes')
    df=pd.DataFrame(columns=columns,data=data_rows,index=index)
    
    df=df.replace('',np.nan)
    
    # convert strings to numerical values where possible
    for col in df.columns:
        if not col[0] in ['#','Date Time']:
            titles=df.columns.get_level_values('title')
            try:
                df[col]=pd.to_numeric(df[col])
            except ValueError:
                pass
    
    return df

df=get_dataframe(hr.header_list,hr.data_rows)
df

title,#,Date Time,Temp,Coupler Detached,Coupler Attached,Host Connected,Stopped,End Of File
timezone_str,NaN,GMT+01:00,NaN,NaN,NaN,NaN,NaN,NaN
units,NaN,NaN,Â°F,NaN,NaN,NaN,NaN,NaN
logger_serial_number,NaN,NaN,10469238,10469238,10469238,10469238,10469238,10469238
sensor_serial_number,NaN,NaN,10469238,NaN,NaN,NaN,NaN,NaN
datetimes,,,,,,,,
2019-10-10 11:00:00+01:00,1,10/10/19 11:00:00 AM,72.199,NaN,NaN,NaN,NaN,NaN
2019-10-10 11:00:02+01:00,2,10/10/19 11:00:02 AM,NaN,Logged,NaN,NaN,NaN,NaN
2019-10-10 11:05:00+01:00,3,10/10/19 11:05:00 AM,72.545,NaN,NaN,NaN,NaN,NaN
2019-10-10 11:10:00+01:00,4,10/10/19 11:10:00 AM,72.372,NaN,NaN,NaN,NaN,NaN
2019-10-10 11:15:00+01:00,5,10/10/19 11:15:00 AM,72.545,NaN,NaN,NaN,NaN,NaN


In [8]:
df.columns[0]

('#', nan, nan, nan, nan)

To access a column:

In [9]:
df['Temp']

timezone_str,NaN
units,Â°F
logger_serial_number,10469238
sensor_serial_number,10469238
datetimes,
2019-10-10 11:00:00+01:00,72.199
2019-10-10 11:00:02+01:00,NaN
2019-10-10 11:05:00+01:00,72.545
2019-10-10 11:10:00+01:00,72.372
2019-10-10 11:15:00+01:00,72.545
2019-10-10 11:20:00+01:00,72.545


In [10]:
df['Temp'].columns.get_level_values('units')[0]

'Â°F'

In [11]:
df['Temp'].columns.get_level_values('logger_serial_number')[0]

'10469238'

## Final version

In [12]:
df=hr.get_dataframe()
df

title,#,Date Time,Temp,Coupler Detached,Coupler Attached,Host Connected,Stopped,End Of File
timezone_str,NaN,GMT+01:00,NaN,NaN,NaN,NaN,NaN,NaN
units,NaN,NaN,Â°F,NaN,NaN,NaN,NaN,NaN
logger_serial_number,NaN,NaN,10469238,10469238,10469238,10469238,10469238,10469238
sensor_serial_number,NaN,NaN,10469238,NaN,NaN,NaN,NaN,NaN
datetimes,,,,,,,,
2019-10-10 11:00:00+01:00,1,10/10/19 11:00:00 AM,72.199,NaN,NaN,NaN,NaN,NaN
2019-10-10 11:00:02+01:00,2,10/10/19 11:00:02 AM,NaN,Logged,NaN,NaN,NaN,NaN
2019-10-10 11:05:00+01:00,3,10/10/19 11:05:00 AM,72.545,NaN,NaN,NaN,NaN,NaN
2019-10-10 11:10:00+01:00,4,10/10/19 11:10:00 AM,72.372,NaN,NaN,NaN,NaN,NaN
2019-10-10 11:15:00+01:00,5,10/10/19 11:15:00 AM,72.545,NaN,NaN,NaN,NaN,NaN
